In [1]:
import pandas as pd 
import json

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

In [2]:
def load_molecular_data(smiles):
    with open(f'molecule_properties/{smiles}.json', 'r') as handle:
        d = json.load(handle)

    # for charges, fukui_electrophilicity, fukui_nucleophilicity, fukui_radical
    # it is a dict. take the min, max, mean of the values of the dict and make a new key/value pairs in the root 
    # of d
    for key in ['charges', 'fukui_electrophilicity', 'fukui_nucleophilicity', 'fukui_radical']:
        d[key + '_min'] = min(d[key].values())
        d[key + '_max'] = max(d[key].values())
        d[key + '_mean'] = sum(d[key].values()) / len(d[key].values())

    d['dipole_x'] = d['dipole'][0]
    d['dipole_y'] = d['dipole'][1]
    d['dipole_z'] = d['dipole'][2]

    return d

In [3]:
def molecular_features(smiles):
    d = load_molecular_data(smiles)
    # select only the keys for which we have float values
    d = {k: v for k, v in d.items() if isinstance(v, float)}
    return d

In [4]:
molecular_features('C_C=C_C')

{'ip': 13.682973015266317,
 'ip_corrected': 8.836973015266317,
 'ea': 1.2218398518750326,
 'homo': -0.3816405868399534,
 'lumo': -0.1874791546885192,
 'global_electrophilicity': 0.27258173104659283,
 'global_nucleophilicity': -8.836973015266317,
 'best_conformer_energy': -12.612394464140023,
 'charges_min': -0.10224603368115683,
 'charges_max': 0.04109184075729364,
 'charges_mean': -1.9024134119878985e-16,
 'fukui_electrophilicity_min': -0.05083731556660036,
 'fukui_electrophilicity_max': 0.15345645443935063,
 'fukui_electrophilicity_mean': 0.08333333333333313,
 'fukui_nucleophilicity_min': -0.026684857821392627,
 'fukui_nucleophilicity_max': 0.13523914792194044,
 'fukui_nucleophilicity_mean': 0.08333333333340077,
 'fukui_radical_min': -0.038761080705294174,
 'fukui_radical_max': 0.11937515617831551,
 'fukui_radical_mean': 0.08333333333336708,
 'dipole_x': -3.718452829648182e-07,
 'dipole_y': -3.899439447074193e-07,
 'dipole_z': -2.845134707431372e-09}

In [5]:
with open('processed_reactions/all_reactions.json', 'r') as file:
    data = json.load(file)

def is_within_deviation(actual_product, expected_product, deviation=0.10):
    if expected_product == 0:
        return actual_product == 0
    return abs(actual_product - expected_product) / abs(expected_product) <= deviation


for entry in data:
    r1 = entry['r_values'].get('constant_1')
    r2 = entry['r_values'].get('constant_2')
    r_product = entry.get('r-product')
    
    if r_product is None:
        entry['r-product_filter'] = False
        continue
    
    actual_product = r1 * r2
    
    # Check for division by zero
    if r_product == 0:
        deviation = float('inf') if actual_product != 0 else 0
    else:
        deviation = abs(actual_product - r_product) / abs(r_product)
    
    if is_within_deviation(actual_product, r_product):
        entry['r-product_filter'] = False
    else:
        entry['r-product_filter'] = True # reaction should be filtered out


def filter_conf_intervals(row):
    if 'conf_intervals' in row and 'constant_conf_1' in row['conf_intervals'] and 'constant_conf_2' in row['conf_intervals']:
        conf_1 = row['conf_intervals']['constant_conf_1']
        conf_2 = row['conf_intervals']['constant_conf_2']
        
        # Ensure 'r1' and 'r2' are correctly retrieved from the row
        r1 = row.get('r_values', {}).get('constant_1')
        r2 = row.get('r_values', {}).get('constant_2')
        
        if r1 is not None and r2 is not None and conf_1 is not None and conf_2 is not None:
            # Filter condition: Confidence intervals should not be greater than the corresponding r-values
            return (conf_1 <= 1 * r1) and (conf_2 <= 1 * r2)
    
    # If conditions are not met, return True by default, meaning the row will not be filtered out
    return True

In [6]:
# Convert JSON data to DataFrame
df_full = pd.DataFrame(data)

print('Initial datapoints: ', len(df_full))
df_full = df_full[df_full.apply(filter_conf_intervals, axis=1)]
print('Datapoints after confidence filter:', len(df_full))

# Separate the filtered data
df_filtered = df_full[df_full['r-product_filter'] == False]
print('Datapoints after r-product filter:', len(df_filtered))

Initial datapoints:  1138
Datapoints after confidence filter: 1060
Datapoints after r-product filter: 1037


In [7]:
df_filtered['r1'] = df_filtered['r_values'].apply(lambda x: x['constant_1'] if isinstance(x, dict) and 'constant_1' in x else None)
df_filtered['r2'] = df_filtered['r_values'].apply(lambda x: x['constant_2'] if isinstance(x, dict) and 'constant_2' in x else None)

df_filtered.dropna(subset=['r1', 'r2',  'monomer1_s', 'monomer2_s', 'temperature', 'calculation_method', 'polymerization_type'], inplace=True)

# if polymerization type is "bulk" then set solvent to "bulk"
df_filtered.loc[df_filtered['polymerization_type'] == 'bulk', 'solvent'] = 'bulk'

df_filtered.dropna(subset=['solvent'], inplace=True)

df_filtered.drop(columns=['r-product_filter', 'r_values', 'r-product', 'monomer1_data', 'monomer2_data', 'conf_intervals'], inplace=True)

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_92265/274317826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['r1'] = df_filtered['r_values'].apply(lambda x: x['constant_1'] if isinstance(x, dict) and 'constant_1' in x else None)
/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_92265/274317826.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['r2'] = df_filtered['r_values'].apply(lambda x: x['constant_2'] if isinstance(x, dict) and 'constant_2' in x else 

In [8]:
df_filtered_flipped = []

# create the same rows, but with flipped monomers, i.e. monomer1_s <-> monomer2_s, monomer1 <-> monomer2, and r1 <-> r2
for index, row in df_filtered.iterrows():
    flipped_row = row.copy()
    flipped_row['monomer1_s'] = row['monomer2_s']
    flipped_row['monomer2_s'] = row['monomer1_s']
    flipped_row['monomer1'] = row['monomer2']
    flipped_row['monomer2'] = row['monomer1']
    flipped_row['r1'] = row['r2']
    flipped_row['r2'] = row['r1']
    df_filtered_flipped.append(flipped_row)

df_filtered_flipped = pd.DataFrame(df_filtered_flipped)
df_filtered_flipped

,file,monomer1_s,monomer2_s,monomer1,monomer2,temperature,temperature_unit,solvent,method,source,calculation_method,polymerization_type,logP,r1,r2
0,paper01.json,C=Cc1ccccc1,C=C(C)C(=O)O,styrene,methacrylic acid,60.0,°C,ClC(Cl)(Cl)Cl,solvent,https://doi.org/10.1002/macp.1985.021860819,Kelen-Tudor,free radical,2.55290,0.06,0.54
1,paper01.json,C=Cc1ccccc1,C=C(C)C(=O)O,styrene,methacrylic acid,60.0,°C,ClC(Cl)Cl,solvent,https://doi.org/10.1002/macp.1985.021860819,Kelen-Tudor,free radical,1.98640,0.08,0.51
2,paper01.json,C=Cc1ccccc1,C=C(C)C(=O)O,styrene,methacrylic acid,60.0,°C,CC(C)=O,solvent,https://doi.org/10.1002/macp.1985.021860819,Kelen-Tudor,free radical,0.59530,0.65,0.43
3,paper01.json,C=Cc1ccccc1,C=C(C)C(=O)O,styrene,methacrylic acid,60.0,°C,C1COCCO1,solvent,https://doi.org/10.1002/macp.1985.021860819,Kelen-Tudor,free radical,0.03320,0.59,0.41
4,paper01.json,C=Cc1ccccc1,C=C(C)C(=O)O,styrene,methacrylic acid,60.0,°C,CC#N,solvent,https://doi.org/10.1002/macp.1985.021860819,Kelen-Tudor,free radical,0.52988,0.29,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,paper_99.json,C=CC#N,C=CC(=O)OCCCCCCCC,Acrylonitrile,Octyl acrylate,60.0,°C,CC(C)(C)O,solvent,https://doi.org/10.1002/app.1960.070040111,Fineman and Ross,free radical,0.77720,0.83,1.93
1134,paper_99.json,C=CC#N,C=CC(=O)OCCCCCCCCCCCCCCCCCC,Acrylonitrile,Octadecyl acrylate,60.0,°C,CC(C)(C)O,solvent,https://doi.org/10.1002/app.1960.070040111,Fineman and Ross,free radical,0.77720,0.68,1.74
1135,paper_99.json,C=C(Cl)Cl,C=CC(=O)OCCCC,Vinylidene chloride,Butyl acrylate,50.0,°C,CC(C)(C)O,solvent,https://doi.org/10.1002/app.1960.070040111,Fineman and Ross,free radical,0.77720,0.83,0.88
1136,paper_99.json,C=C(Cl)Cl,C=CC(=O)OCCCCCCCC,Vinylidene chloride,Octyl acrylate,50.0,°C,CC(C)(C)O,solvent,https://doi.org/10.1002/app.1960.070040111,Fineman and Ross,free radical,0.77720,0.70,0.87


In [9]:
from rdkit import Chem
from rdkit.Chem.Descriptors import MolLogP

In [10]:
# now, add the features for each monomer by loading the corresponding JSON file with `molecular_features` based on the monomer{i}_s column
def add_molecular_features(df): 
    new_rows = []
    for index, row in df.iterrows():
        try:
            monomer1_s = row['monomer1_s']
            monomer2_s = row['monomer2_s']
            monomer1 = row['monomer1']
            monomer2 = row['monomer2']
            temperature = row['temperature']
            solvent = row['solvent']
            calculation_method = row['calculation_method']
            polymerization_method = row['polymerization_type']
            if solvent == 'bulk':
                solvent_logp = 0
            else:
                solvent_logp = MolLogP(Chem.MolFromSmiles(solvent))

            monomer1_data = molecular_features(monomer1_s)
            monomer2_data = molecular_features(monomer2_s)
            
            # add _1 to the keys of monomer1_data and _2 to the keys of monomer2_data
            monomer1_data = {f'{k}_1': v for k, v in monomer1_data.items()}
            monomer2_data = {f'{k}_2': v for k, v in monomer2_data.items()}

            # now, create new dict with all the data 
            new_row = {**row, **monomer1_data, **monomer2_data, 'temperature': temperature, 'solvent': solvent, 'calculation_method': calculation_method, 'polymerization_method': polymerization_method, 'solvent_logp': solvent_logp}

            new_rows.append(new_row)
        except FileNotFoundError as e: 
            print(f"File not found: {e}")
    return pd.DataFrame(new_rows) 

In [11]:
df_filtered = add_molecular_features(df_filtered)
df_filtered_flipped = add_molecular_features(df_filtered_flipped)

File not found: [Errno 2] No such file or directory: 'molecule_properties/C=Cc1cc(O)ccc1O.O=C(O)c1ccccc1.O=C(O)c1ccccc1.json'
File not found: [Errno 2] No such file or directory: 'molecule_properties/C=Cc1cc(O)ccc1O.O=C(O)c1ccccc1.O=C(O)c1ccccc1.json'
File not found: [Errno 2] No such file or directory: 'molecule_properties/C/C=C/C.json'
File not found: [Errno 2] No such file or directory: 'molecule_properties/C/C=C/C.json'
File not found: [Errno 2] No such file or directory: 'molecule_properties/C/C=C\\C.json'
File not found: [Errno 2] No such file or directory: 'molecule_properties/C/C=C\\C.json'
File not found: [Errno 2] No such file or directory: 'molecule_properties/C=CC(=O)[O-].C=CC(=O)[O-].[Zn+2].json'
File not found: [Errno 2] No such file or directory: 'molecule_properties/O=C(O)/C=C/C=C/C(=O)O.json'
File not found: [Errno 2] No such file or directory: 'molecule_properties/O=C(O)/C=C/C=C/C(=O)O.json'
File not found: [Errno 2] No such file or directory: 'molecule_properties/CCO

In [12]:
len(df_filtered)

539

In [14]:
df_filtered[df_filtered['method']=='bulk']

,file,monomer1_s,monomer2_s,monomer1,monomer2,temperature,temperature_unit,solvent,method,source,...,fukui_nucleophilicity_max_2,fukui_nucleophilicity_mean_2,fukui_radical_min_2,fukui_radical_max_2,fukui_radical_mean_2,dipole_x_2,dipole_y_2,dipole_z_2,polymerization_method,solvent_logp
66,paper_169.json,C=Cc1ccccc1,C=Cc1cccc2c1[nH]c1ccccc12,Styrene,Vinyl Carbazole,70.0,°C,c1ccccc1,bulk,"Journal of Polymer Science Vol. IV, pp. 215-21...",...,0.067678,0.038462,0.012596,0.063445,0.038462,-0.113567,0.614863,0.068897,free radical,1.6866
67,paper_169.json,C=C(C)C(=O)OC,C=Cc1cccc2c1[nH]c1ccccc12,Methyl Methacrylate,Vinyl Carbazole,70.0,°C,c1ccccc1,bulk,"Journal of Polymer Science Vol. IV, pp. 215-21...",...,0.067678,0.038462,0.012596,0.063445,0.038462,-0.113567,0.614863,0.068897,free radical,1.6866
68,paper_169.json,C=Cc1ccccc1,C=C(Cl)CCl,Styrene,"1,2-Dichloro-2-propene",70.0,°C,c1ccccc1,bulk,"Journal of Polymer Science Vol. IV, pp. 215-21...",...,0.318609,0.111111,-0.035264,0.282111,0.111111,-0.373286,0.006694,0.833315,free radical,1.6866
69,paper_169.json,C=C(C)C(=O)OC,C=C(Cl)CCl,Methyl Methacrylate,"1,2-Dichloro-2-propene",70.0,°C,c1ccccc1,bulk,"Journal of Polymer Science Vol. IV, pp. 215-21...",...,0.318609,0.111111,-0.035264,0.282111,0.111111,-0.373286,0.006694,0.833315,free radical,1.6866
82,paper_181.json,C=Cc1ccccc1,C=Cc1ccc(C)nc1,Styrene,2-Methyl-5-vinylpyridine,60.0,°C,bulk,bulk,https://doi.org/10.1002/pol.1959.120391203,...,0.174614,0.055556,-0.023113,0.130269,0.055556,0.127251,0.678507,-0.079675,bulk,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,paper_776.json,C=Cc1ccccc1,C=CSC,Styrene,Methyl Vinyl Sulfide,60.0,°C,O,bulk,https://doi.org/10.1021/ja01157a003,...,0.550493,0.100000,-0.066296,0.487713,0.100000,0.372830,0.283551,0.718924,free radical,-0.8247
500,paper_776.json,C=COC(C)=O,C=CSC,Vinyl Acetate,Methyl Vinyl Sulfide,60.0,°C,O,bulk,https://doi.org/10.1021/ja01157a003,...,0.550493,0.100000,-0.066296,0.487713,0.100000,0.372830,0.283551,0.718924,free radical,-0.8247
507,paper_795.json,C=COC(C)=S,C=CN1C(=O)CCC1=O,Vinyl thioacetate,N-Vinylsuccinimide,60.0,°C,c1ccccc1,bulk,https://doi.org/10.1007/BF00549745,...,0.155082,0.062500,-0.011413,0.157335,0.062500,-0.813073,0.208070,-0.051568,free radical,1.6866
508,paper_795.json,C=COC(C)=S,C=CN1C(=O)c2ccccc2C1=O,Vinyl thioacetate,N-Vinylphthalimide,60.0,°C,c1ccccc1,bulk,https://doi.org/10.1007/BF00549745,...,0.108985,0.050000,0.010853,0.123950,0.050000,1.198961,-0.243353,-0.002086,free radical,1.6866


## Modeling

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

# Define the features
numerical_features = ['temperature', 'ip_corrected_1', 'ea_1', 'homo_1', 'lumo_1',
                      'global_electrophilicity_1', 'global_nucleophilicity_1',
                      'best_conformer_energy_1', 'charges_min_1', 'charges_max_1',
                      'charges_mean_1', 'fukui_electrophilicity_min_1',
                      'fukui_electrophilicity_max_1', 'fukui_electrophilicity_mean_1',
                      'fukui_nucleophilicity_min_1', 'fukui_nucleophilicity_max_1',
                      'fukui_nucleophilicity_mean_1', 'fukui_radical_min_1',
                      'fukui_radical_max_1', 'fukui_radical_mean_1', 'dipole_x_1',
                      'dipole_y_1', 'dipole_z_1',  'ip_corrected_2', 'ea_2', 'homo_2',
                      'lumo_2', 'global_electrophilicity_2', 'global_nucleophilicity_2',  
                      'charges_min_2', 'charges_max_2', 'charges_mean_2', 
                      'fukui_electrophilicity_min_2', 'fukui_electrophilicity_max_2', 
                      'fukui_electrophilicity_mean_2', 'fukui_nucleophilicity_min_2', 
                      'fukui_nucleophilicity_max_2', 'fukui_nucleophilicity_mean_2', 
                      'fukui_radical_min_2', 'fukui_radical_max_2', 'fukui_radical_mean_2', 
                      'dipole_x_2', 'dipole_y_2', 'dipole_z_2', 'solvent_logp']

categorical_features = ['temperature_unit', 'solvent', 'calculation_method', 
                        'polymerization_type', 'polymerization_method']

# Define the transformer
transformer = ColumnTransformer([
    ('numerical', StandardScaler(), numerical_features),
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Define the power transformer for the label
pt = PowerTransformer(method='yeo-johnson')

# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'n_estimators': [100, 500, 1000, 5000],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': np.logspace(-3, 0, 10),
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': np.linspace(0, 1, 10),
    'reg_lambda': np.linspace(0, 1, 10)
}

# Implement K-Fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold_scores = []

for fold, (train_idx, test_idx) in enumerate(kf.split(df_filtered), 1):
    print(f"Fold {fold}")
    
    # Split the data
    train = df_filtered.iloc[train_idx]
    test = df_filtered.iloc[test_idx]
    
    train_flipped = df_filtered_flipped.iloc[train_idx]
    test_flipped = df_filtered_flipped.iloc[test_idx]
    train = pd.concat([train, train_flipped])
    test = pd.concat([test, test_flipped])
    
    # Create the target variable
    train['r1r2'] = train['r1'] * train['r2']
    test['r1r2'] = test['r1'] * test['r2']
    
    # Fit and transform the features
    X_train = transformer.fit_transform(train)
    X_test = transformer.transform(test)
    
    # Transform the label
    y_train = pt.fit_transform(train['r1r2'].values.reshape(-1, 1)).ravel()
    y_test = pt.transform(test['r1r2'].values.reshape(-1, 1)).ravel()
    
    # Define the model
    model = XGBRegressor(random_state=42)
    
    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(model, param_distributions=param_grid, 
                                       n_iter=100, cv=3, verbose=1, 
                                       random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train, 
                    #   early_stopping_rounds=10, 
                      eval_set=[(X_test, y_test)],
                      verbose=False)
    
    # Get the best model
    best_model = random_search.best_estimator_
    
    # Make predictions
    y_pred_train = best_model.predict(X_train)
    y_pred_test = best_model.predict(X_test)
    
    # Calculate metrics
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    
    print(f"Train MSE: {mse_train:.4f}, Train R2: {r2_train:.4f}")
    print(f"Test MSE: {mse_test:.4f}, Test R2: {r2_test:.4f}")
    
    fold_scores.append((r2_train, r2_test))

# Calculate average scores across all folds
avg_train_r2 = np.mean([score[0] for score in fold_scores])
avg_test_r2 = np.mean([score[1] for score in fold_scores])

print(f"\nAverage Train R2: {avg_train_r2:.4f}")
print(f"Average Test R2: {avg_test_r2:.4f}")

Fold 1
Fitting 3 folds for each of 100 candidates, totalling 300 fits


Train MSE: 0.0055, Train R2: 0.9945
Test MSE: 0.7059, Test R2: 0.4086
Fold 2
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Train MSE: 0.0073, Train R2: 0.9927
Test MSE: 0.3173, Test R2: 0.7044
Fold 3
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Train MSE: 0.2393, Train R2: 0.7607
Test MSE: 0.7002, Test R2: 0.3241
Fold 4
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Train MSE: 0.1139, Train R2: 0.8861
Test MSE: 0.3639, Test R2: 0.5711
Fold 5
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Train MSE: 0.0071, Train R2: 0.9929
Test MSE: 0.4224, Test R2: 0.5184
Fold 6
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Train MSE: 0.0072, Train R2: 0.9928
Test MSE: 0.4426, Test R2: 0.5192
Fold 7
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Train MSE: 0.0026, Train R2: 0.9974
Test MSE: 0.5627, Test R2: 0.3928
Fold 8
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Train MSE: 0.1194, T